<a href="https://colab.research.google.com/github/pkhlingam09/Fashion-Search-AI/blob/Final_Done_0405/Fashion_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Dataset Link (Kaggle)
# https://www.kaggle.com/datasets/djagatiya/myntra-fashion-product-dataset

In [ ]:
# import os

# def find_file(root_dir, file_name):
#     for dirpath, dirnames, filenames in os.walk(root_dir):
#         if file_name in filenames:
#             return os.path.join(dirpath, file_name)
#     return None

# file_path = find_file("/", "Fashion Dataset v2.csv") #Searches from the root directory

# if file_path:
#   print(file_path)
# else:
#   print("File not found")

/content/drive/MyDrive/Fashion_Search_AI/Fashion Dataset v2.csv


In [ ]:
#############################################################################################################################################################################################################################
###################################################################################################   MYNTRA ASSIGNMENT   ###################################################################################################

In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/Fashion_Search_AI/"

def read_dataset():
  !pip install -q kaggle chromadb
  files.upload()
  !mkdir -p ~/.kaggle
  !cp kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json
  !cat ~/.kaggle/kaggle.json
  !kaggle datasets download "djagatiya/myntra-fashion-product-dataset" -p file_path
  !unzip file_path+"myntra-fashion-product-dataset.zip" -d file_path

In [4]:
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.9 MB/s eta 0:0

In [5]:
import os
import pathlib

import string
import re
import ast
import json
import numpy as np
import pandas as pd

import chromadb

from openai import OpenAI
from sentence_transformers import CrossEncoder, util

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

stop_word = stopwords.words("english")

In [ ]:
# pd.set_option('display.max_rows', 200)
# pd.set_option('display.max_columns', 200)

In [ ]:
gpt_model = "gpt-4o-mini"
moderator = "omni-moderation-latest"

In [ ]:
df = pd.read_csv(file_path + "Fashion Dataset v2.csv")

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
round(100 * df.isna().sum()/df.shape[0], 3)

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
class clean_data:

  def __init__(self, df):
    clean_description(df)
    clean_p_attributes(df)
    remove_chosen_keys(df)
    merge_cols(df)

  ## clean description column
  def clean_description(self, df):
    df["description"] = df["description"].apply(lambda x: x.lower())
    htmltags = re.compile("(<.*?>)|&nbsp;|&amp;|[;|]+")
    df["description"] = df["description"].apply(lambda x: re.sub(htmltags, ", ", x))
    puncts = re.compile(r"[ \(\)]+")
    df["description"] = df["description"].apply(lambda x: re.sub(puncts, " ", x))
    s_quote = re.compile(r"[',]{2,}|(, ,)+")
    df["description"] = df["description"].apply(lambda x: re.sub(s_quote, "", x))
    df["description"] = df["description"].apply(lambda x: re.sub("(?i)(size s)", "size s, ", x))
    df["description"] = df["description"].apply(lambda x: re.sub("(?i)(size m)", "size m, ", x))
    df["description"] = df["description"].apply(lambda x: re.sub("(?i)(size l)", "size l, ", x))
    spaces = re.compile(r" {2,}")
    df["description"] = df["description"].apply(lambda x: re.sub(spaces, " ", x).strip())
    df["description"] = df["description"].apply(lambda x: re.sub(" *,$", "", x))

  ## Clean p_attributes column
  def clean_p_attributes(self, df):
    df["p_attributes"] = df["p_attributes"].apply(lambda x: x.lower())
    htmltags = re.compile(r"(<.*?>)?(\\r|\\n)")
    df["p_attributes"] = df["p_attributes"].apply(lambda x: re.sub(htmltags, " ", x))
    re.sub(r"(<.*?>)?(\\r|\\n)", " ", df.loc[956, "p_attributes"])
    df['p_attributes'] = df['p_attributes'].apply(lambda x: ast.literal_eval(x))

  ## Remove Chosen keys from dictionaries
  def remove_chosen_keys(self, df):
    pattr_keys_toremove = ["body shape id", "body or garment size"]
    for ind, dicts in enumerate(df["p_attributes"].copy()):
      for key in pattr_keys_toremove:
        if key in dicts.keys():
          del dicts[key]
      df.loc[ind, "p_attributes"] = dicts

  ## Merge columns Product name, Products, price with description column
  def merge_cols(self, df):
    for ind in range(0, df.shape[0]):
      df.loc[ind, "description"] = f"Product name is {df['name'][ind]}. Products available are {df['products'][ind]} for a total price of {df['price'][ind]}. Product colour is {df['colour'][ind]}. Brand is {df['brand'][ind]}. " + df.loc[ind, "description"]


In [ ]:
## Find length for each row of description column
df["pattr_len"] = df.loc[:10, 'p_attributes'].apply(lambda x: len(x.split()))

In [ ]:
df["desc_len"] = df['description'].apply(lambda x: len(x.split()))

In [ ]:
df.head(5)

In [ ]:
df['p_id'] = df['p_id'].astype("str")

In [ ]:
myntra = df.drop(["ratingCount", "avg_rating", "name", "products", "colour", "brand", "price"], axis=1)

In [ ]:
myntra = myntra[["p_id", "description", "p_attributes"]]

In [ ]:
# ____________________________________________________________________________________________  EDA and Data Cleaning Complete  ____________________________________________________________________________________________
# __________________________________________________________________________________________________________________________________________________________________________________________________________________________
# __________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [ ]:
## Create Embeddings using ChromaDB

In [ ]:
embed_model = "text-embedding-3-large"
cross_encode_model = "cross-encoder/ms-marco-MiniLM-L-6-v2"
api_file = file_path + "api_key.txt"
threshold = 0.15

In [ ]:
client = chromadb.PersistentClient(path= file_path + "chroma_db")

In [ ]:
class OpenAI_Embeddings:

  def __init__(self, model, api_path):
    self.model = model
    self.api_path = api_path
    self.embedding_function = None
    self.myntra_collections = None
    self.create_my_embedding()

  def get_api(self):
    with open(self.api_path, "r") as fptr:
      api_key = fptr.read()
      fptr.close()
    return api_key

  def create_my_embedding(self):
    self.embedding_function = OpenAIEmbeddingFunction(
                                                        api_key = self.get_api(),
                                                        model_name = self.model
                                                     )

  def create_get_collection(self, name):
    ## Create or Load Chroma Collection
    self.myntra_collections = client.get_or_create_collection(
                                                            name = name,
                                                            embedding_function = self.embedding_function
                                                        )
    return self.myntra_collections

  def add_to_collection(self, data, collections):
    ## Add data to Chroma Collection
    prev = 0
    docs_list = data["description"].to_list()
    meta_list = data['p_attributes'].to_list()
    id_list = data['p_id'].to_list()
    ## Add data to Chroma Collection
    for batch_size in range(1000, len(docs_list), 1000):
      collections.add(
                                    documents = docs_list[prev:batch_size],
                                    metadatas = meta_list[prev:batch_size],
                                    ids = id_list[prev:batch_size]
                                 )
      prev = batch_size
    if prev < len(docs_list):
      collections.add(
                                     documents = docs_list[prev:],
                                     metadatas = meta_list[prev:],
                                     ids = id_list[prev:]
                                 )
    return collections

In [ ]:
class Chroma_Search(OpenAI_Embeddings):

  def __init__(self, model, api_path):
    super().__init__(model, api_path)

  def text_query(self, collections, query, w_clause = "", w_doc_clause = ""):
    query_results = collections.query(
                                          query_texts = [query],
                                          n_results = 10
                                     )
    return query_results

In [ ]:
## This embedding model is to create embeddings and query
myntra_obj = Chroma_Search(embed_model, api_file)
myntra_collection = myntra_obj.create_get_collection("myntra")

if not os.path.isdir(file_path + "chroma_db"):
  myntra_obj.add_to_collection(myntra, myntra_collection)

myntra_collection.peek()

In [ ]:
## This embedding model is to create cache and query
cache_obj = Chroma_Search(embed_model, api_file)
cache_collection = cache_obj.create_get_collection("cache_myntra")

cache_collection.peek()

In [ ]:
def user_query(query, attr_dict, w_doc_clause):
  ids = []
  documents = []
  distances = []
  metadatas = []
  results_df = pd.DataFrame()


  cache_result = cache_obj.text_query(cache_collection, query, attr_dict, w_doc_clause)

  if cache_result['distances'][0] == []: # if cache is empty
    result = myntra_obj.text_query(myntra_collection, query, attr_dict, w_doc_clause)

    keys = []
    vals = []
    for key, val in result.items():
      if val == None:
        continue
      if key.lower() != "embeddings" and key.lower() != "uris" and key.lower() != "data" and key.lower() != "included":
        for i in range(10):
              keys.append(str(key)+str(i))
              vals.append(str(val[0][i]))
      ## Add new query to collection
        cache_collection.add(
                              documents = [query],
                              ids = [query],
                              metadatas =  dict(zip(keys, vals))
                            )
        ## Create Result database
        result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
        results_df = pd.DataFrame.from_dict(result_dict)
        results_df
  else:
    cache_result_dict = cache_result['metadatas'][0][0]

    # Loop through each inner list and then through the dictionary
    for key, value in cache_result_dict.items():
        if 'ids' in key:
            ids.append(value)
        elif 'documents' in key:
            documents.append(value)
        elif 'distances' in key:
            distances.append(value)
        elif 'metadatas' in key:
            metadatas.append(value)

    # Create a DataFrame
    results_df = pd.DataFrame({
      'IDs': ids,
      'Documents': documents,
      'Distances': distances,
      'Metadatas': metadatas
    })
    results_df

In [ ]:
 ## Cross-Encoders

In [ ]:
def crossencode_user_query(df):
  cross_encoder = CrossEncoder(cross_encode_model)
  scores = cross_encoder.predict([[query, response] for response in results_df['Documents']])
  results_df['cross_encoder_scores'] = scores
  results_df.sort_values(by='cross_encoder_scores', ascending=False, inplace=True)
  return results_df[["Metadatas", "Documents"]]

In [ ]:
## Prompt Generation using OpenAI

In [ ]:
chat_client = OpenAI(api_key=myntra_obj.get_api())

In [ ]:
def fashion_converse(conversation):
    chat_resp = chat_client.chat.completions.create(
                                    model = gpt_model,
                                    messages = conversation,
                                    temperature = 0.4,
                                    max_tokens = 200,
                                    tools = tools_shop_assist(),
                                    tool_choice = "auto"
                                    )
    return chat_resp.choices[0].message

def chat_moderator(msg):
    response = chat_client.moderations.create(
                                    model = moderator,
                                    input = msg
                                    )

    return response.results[0].flagged

In [ ]:
def list_products(inp_query_dict):
  w_doc_clause = ""
  inp_query = inp_query_dict["inp_query"]
  attr_dict = ast.literal_eval(inp_query_dict["attr_dict"])
  if "colour" in list(attr_dict.keys()):
    w_doc_clause = {"colour": attr_dict['colour'].lower()}
    del attr_dict['colour']
  if "size" in list(attr_dict.keys()):
    w_doc_clause = {"size": attr_dict['size'].lower()}
    del attr_dict['size']
  df = user_query(query, attr_dict, w_doc_clause)
  return crossencode_user_query(df)

In [ ]:
## Belongs in Tools_Calls Class

def tools_shop_assist():
        criteria = [
                        {
                            "type": "function",
                            "function": {
                                "name": "list_products",
                                "description": "Function takes input string and additional criteria as input and returns ",
                                "strict": False,
                                "parameters": {
                                    "type": "object",
                                    "properties": {
                                        "inp_query": {
                                            "type": "string",
                                            "description": "string indicating input given by user eg. find all the skirts in pink.",
                                        },
                                        "attr_dict": {
                                            "type": "string",
                                            "description": "features the apparel needs to have. eg. {'size': 'long', 'colour': 'pink', 'design': 'floral'}.",
                                        },
                                    },
                                    "required": ["inp_query", "attr_dict"],
                                    "additionalProperties": False
                                },
                            },
                        }
                    ]

        return criteria


In [ ]:
def start_conversations():
  converse = initial_conversation()
  chat_resp = fashion_converse(converse)
  converse.append({"role": "assistant", "content": str(chat_resp.content)})
  print(chat_resp.content)

  while True:
    user_input = input().lower().strip()
    if user_input == "exit":
      break

    is_flagged = chat_moderator(user_input)
    if is_flagged:
        print("Sorry, this message has been flagged. Cannot be accepted.")
        continue ## Skip everything and get back to input()

    converse.append({"role": "user", "content": user_input})
    chat_resp = fashion_converse(converse)
    if chat_resp.tool_calls:
      for tool_call in chat_resp.tool_calls:
        func_name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        tool_response = list_products(args)
        ## Append Function Calling Request
        converse.append(chat_resp)
        converse.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": func_name,
                        "content": str(tool_response)
                        })
      chat_resp = fashion_converse(converse)
      print(chat_resp.content)
    else:
      converse.append({"role": "assistant", "content": str(chat_resp.content)})
      print(chat_resp.content)


In [ ]:
def initial_conversation():
    delimiters = "#####"

    chat_prompt = f"""
                      You new an expert in Fashion Digital Marketer, Fashion Merchandise, Stylist and E-commerce Consultant.Your job is to assist customers find the apparel they want to buy through polite and professional conversation.
                      {delimiters}
                      When the customer requests to find a particular apparel or a combination of apparels, try to understand what apparel or combination the customer is requesting and \n
                      search online to find all the additional criteria needed to know from the customer about the apparel to narrow down the list.
                      Showing the internal information collection, summary and format will be penalised. only the conversation and the answers given to customer should be displayed.
                      Once the additional criteria are identified ask a few questions to get the answers from customer for atleast two criteria. Record any response given by customer for the criteria.
                      Once the data is collected from customer, ask the customer (if he wants to add any other information. if yes allow customer to add details. if the criteria is not clear ask customer to inform the criteria). else proceed.
                      Once the data is collected from customer, convert the criteria and the answers received for the criteria into a dictionary format.
                      Call the function list_products() with arguments as the complete input string by customer and additional criteria and answers stored in dictionary earlier.
                      list_products() function returns a dataframe, which consists a product in every row. Take product name, products available and total price in rupees details for each product in each row and display them as it is done in ecommerce websites.
                      {delimiters}
                   """
    chat_prompt = [{"role": "system", "content": chat_prompt},
                   {"role": "user", "content": "Start conversation with a polite welcome"}]

    return chat_prompt


In [ ]:
start_conversations()